# Introducción a Problemas de Clasificación en R

En este notebook, exploraremos problemas de clasificación utilizando R. Crearemos un conjunto de datos sintético con dos predictores y una variable categórica a predecir. Visualizaremos los datos con `ggplot2`, explicaremos la función sigmoide, la función de pérdida y su gradiente. Finalmente, implementaremos modelos de clasificación utilizando **regresión logística**, **KNN** y **árboles de decisión** con la biblioteca **caret**.

## Contenido

1. [Generación del Dataset](#1)
2. [Visualización de los Datos](#2)
3. [Función Sigmoide](#3)
4. [Función de Pérdida y Gradiente](#4)
5. [Modelos de Clasificación con caret](#5)
    - 5.1 [Regresión Logística](#5.1)
    - 5.2 [K-Nearest Neighbors (KNN)](#5.2)
    - 5.3 [Árboles de Decisión (rpart)](#5.3)

<a name="1"></a>
## 1. Generación del Dataset

Crearemos un conjunto de datos sintético con dos variables predictoras (`x1` y `x2`) y una variable de clase binaria (`clase`).

In [ ]:
library(modules)
library(tidyverse)
library(lubridate)
library(caret)
library(ggplot2)
library(dplyr)

datos <- read.csv("../sources/circles.csv")

datos <- data.frame(datos[3:nrow(datos), ])

datos$clase <- as.factor(datos$Class)
datos$Class <- NULL
datos$x <- as.double(datos$x)
datos$y <- as.double(datos$y)
datos$x_square <- datos$x^2
datos$y_square <- datos$y^2
x_mean <- mean(datos$x)
y_mean <- mean(datos$y)
datos$radius <- sqrt(datos$x_square + datos$y_square)
datos$distance_to_mean <- sqrt((datos$x - x_mean)^2 + (datos$y - y_mean)^2)
datos$product <- datos$x * datos$y


head(datos)

summary(datos)


<a name="2"></a>
## 2. Visualización de los Datos

Utilizaremos `ggplot2` para visualizar los datos y entender la distribución de las clases.

In [ ]:
# Crear el gráfico
ggplot(datos, aes(x = x, y = y, color = clase)) +
  geom_point() +
  labs(title = "Distribución de las Clases", x = "x", y = "y") +
  theme_minimal()


<a name="3"></a>
## 3. Función Sigmoide

La función sigmoide es utilizada en regresión logística para transformar valores reales en probabilidades entre 0 y 1.

La función sigmoide se define como:

$$
S(z) = \frac{1}{1 + e^{-z}}
$$

Donde $z$ es una combinación lineal de las variables de entrada.Para nuestro caso, una función lineal: $z=\beta_{0} + \beta_{1}x_{1} + ... + \beta_{n}x_{n}$

In [ ]:
# Definir la función sigmoide
sigmoide <- function(z) {
  1 / (1 + exp(-z))
}

# Crear una secuencia de valores
z <- seq(-10, 10, length.out = 100)

# Calcular S(z)
S_z <- sigmoide(z)

# Graficar la función sigmoide
plot(z, S_z,
  type = "l", col = "blue", lwd = 2,
  main = "Función Sigmoide",
  xlab = "z", ylab = "S(z)"
)


<a name="4"></a>
## 4. Función de Pérdida y Gradiente

En la regresión logística, utilizamos la **función de pérdida logística** para medir el error entre las predicciones y las etiquetas reales.

La función de pérdida para una observación es:

$$
L(y, \hat{y}) = -[y \log(\hat{y}) + (1 - y) \log(1 - \hat{y})]
$$

Donde:
- $y$ es la etiqueta real (0 o 1).
- $\hat{y}$ es la probabilidad predicha.

El **gradiente** de la función de pérdida con respecto a los pesos es utilizado para actualizar los pesos durante el entrenamiento:

$\nabla L = (\hat{y} - y)x$

In [ ]:
# Función de pérdida logística
perdida_logistica <- function(y, y_hat) {
  -(y * log(y_hat) + (1 - y) * log(1 - y_hat))
}

# Función para calcular el gradiente
gradiente <- function(y, y_hat, x) {
  (y_hat - y) * x
}

# Ejemplos de valores\,
# Supongamos y = 1 y y_hat variando entre 0.01 y 0.99\n",
y <- 1
y_hat_vals <- seq(0.01, 0.99, by = 0.01)
loss_vals <- perdida_logistica(y, y_hat_vals)

# Graficar la pérdida en función de y_hat\n",
plot(y_hat_vals, loss_vals,
  type = "l", col = "red", lwd = 2,
  main = "Pérdida Logística vs y_hat (y = 1)",
  xlab = "y_hat", ylab = "Pérdida"
)

# Calcular el gradiente para un valor de x
x <- c(1, 2) # Vector de características de ejemplo
gradientes <- sapply(y_hat_vals, function(yh) gradiente(y, yh, x))

# Graficar el gradiente de la primera característica\n",
plot(y_hat_vals, gradientes[1, ],
  type = "l", col = "blue", lwd = 2,
  main = "Gradiente vs y_hat (Componente x1)",
  xlab = "y_hat", ylab = "Gradiente"
)


<a name="5"></a>
## 5. Modelos de Clasificación con caret

Utilizaremos la biblioteca **caret** para entrenar modelos de clasificación y evaluar su rendimiento.

### Preparación de los Datos

Dividiremos los datos en conjuntos de entrenamiento y prueba.

In [ ]:
# Dividir los datos en entrenamiento y prueba
set.seed(123)
indice_train <- createDataPartition(datos$clase, p = 0.7, list = FALSE)
datos_train <- datos[indice_train, ]
datos_test <- datos[-indice_train, ]
# Verificar el tamaño de los conjuntos
dim(datos_train)
dim(datos_test)


<a name="5.1"></a>
### 5.1 Regresión Logística

Entrenaremos un modelo de regresión logística utilizando **caret**.

In [ ]:
control <- trainControl(method = "none")

modelo_logistico_original <- train(clase ~ x + y,
    data = datos_train,
    method = "glm",
    family = "binomial",
    trControl = control
)

summary(modelo_logistico_original)

modelo_logistico_completo <- train(clase ~ .,
    data = datos_train,
    method = "glm",
    family = "binomial",
    trControl = control
)

summary(modelo_logistico_completo)


#### Evaluación del Modelo

In [ ]:
pred_logistico_original <- predict(modelo_logistico_original, newdata = datos_test)
confusionMatrix(pred_logistico_original, datos_test$clase)


pred_logistico_completo <- predict(modelo_logistico_completo, newdata = datos_test)
confusionMatrix(pred_logistico_completo, datos_test$clase)


In [ ]:
# Crear el gráfico
datos_test$prediccion_original <- pred_logistico_original
datos_test$prediccion_completo <- pred_logistico_completo

ggplot(datos_test, aes(x = x, y = y, color = clase, shape = pred_logistico_original)) +
  geom_point() +
  labs(title = "Distribución de las Clases", x = "x", y = "y") +
  theme_minimal()

  ggplot(datos_test, aes(x = x, y = y, color = clase, shape = pred_logistico_completo)) +
  geom_point() +
  labs(title = "Distribución de las Clases", x = "x", y = "y") +
  theme_minimal()


In [ ]:
precision_original <- posPredValue(pred_logistico_original, datos_test$clase, positive = "C1")
recall_original <- sensitivity(pred_logistico_original, datos_test$clase, positive = "C1")

F1_original <- (2 * precision_original * recall_original) / (precision_original + recall_original)

print(precision_original)
print(recall_original)
print(F1_original)

precision_completo <- posPredValue(pred_logistico_completo, datos_test$clase, positive = "C1")
recall_completo <- sensitivity(pred_logistico_completo, datos_test$clase, positive = "C1")

F1_completo <- (2 * precision_completo * recall_completo) / (precision_completo + recall_completo)

print(precision_completo)
print(recall_completo)
print(F1_completo)


<a name="5.2"></a>
### 5.2 K-Nearest Neighbors (KNN)

Entrenaremos un modelo KNN utilizando **caret**.

In [ ]:
# Definir una cuadrícula de hiperparámetros
grid_knn <- expand.grid(k = 5)

# Entrenar el modelo
modelo_knn_original <- train(clase ~ x + y,
    data = datos_train,
    method = "knn",
    trControl = control,
    tuneGrid = grid_knn
)

summary(modelo_knn_original)

modelo_knn_completo <- train(clase ~ .,
    data = datos_train,
    method = "knn",
    trControl = control,
    tuneGrid = grid_knn
)

summary(modelo_knn_completo)


#### Evaluación del Modelo

In [ ]:
pred_knn_original <- predict(modelo_knn_original, newdata = datos_test)
confusionMatrix(pred_knn_original, datos_test$clase)

pred_knn_completo <- predict(modelo_knn_completo, newdata = datos_test)
confusionMatrix(pred_knn_completo, datos_test$clase)


In [ ]:
precision_original <- posPredValue(pred_knn_original, datos_test$clase, positive = "C1")
recall_original <- sensitivity(pred_knn_original, datos_test$clase, positive = "C1")

F1_original <- (2 * precision_original * recall_original) / (precision_original + recall_original)

print(precision_original)
print(recall_original)
print(F1_original)

precision_completo <- posPredValue(pred_knn_completo, datos_test$clase, positive = "C1")
recall_completo <- sensitivity(pred_knn_completo, datos_test$clase, positive = "C1")

F1_completo <- (2 * precision_completo * recall_completo) / (precision_completo + recall_completo)

print(precision_completo)
print(recall_completo)
print(F1_completo)


<a name="5.3"></a>
### 5.3 Árboles de Decisión (rpart)

Entrenaremos un árbol de decisión utilizando **caret**.

In [ ]:
modelo_arbol_original <- train(clase ~ x + y,
    data = datos_train,
    method = "rpart"
)

modelo_arbol_completo <- train(clase ~ .,
    data = datos_train,
    method = "rpart"
)


#### Evaluación del Modelo

In [ ]:
pred_arbol_original <- predict(modelo_arbol_original, newdata = datos_test)
confusionMatrix(pred_arbol_original, datos_test$clase)

pred_arbol_completo <- predict(modelo_arbol_completo, newdata = datos_test)
confusionMatrix(pred_arbol_completo, datos_test$clase)


In [ ]:
precision_original <- posPredValue(pred_arbol_original, datos_test$clase, positive = "C1")
recall_original <- sensitivity(pred_arbol_original, datos_test$clase, positive = "C1")

F1_original <- (2 * precision_original * recall_original) / (precision_original + recall_original)

print(precision_original)
print(recall_original)
print(F1_original)

precision_completo <- posPredValue(pred_arbol_completo, datos_test$clase, positive = "C1")
recall_completo <- sensitivity(pred_arbol_completo, datos_test$clase, positive = "C1")

F1_completo <- (2 * precision_completo * recall_completo) / (precision_completo + recall_completo)

print(precision_completo)
print(recall_completo)
print(F1_completo)


In [ ]:
# Crear el gráfico
datos_test$prediccion_original <- pred_logistico_original
datos_test$prediccion_completo <- pred_logistico_completo

ggplot(datos_test, aes(x = x, y = y, color = clase, shape = pred_logistico_original)) +
  geom_point() +
  labs(title = "Distribución de las Clases", x = "x", y = "y") +
  theme_minimal()

  ggplot(datos_test, aes(x = x, y = y, color = clase, shape = pred_logistico_completo)) +
  geom_point() +
  labs(title = "Distribución de las Clases", x = "x", y = "y") +
  theme_minimal()
